In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

# 경로 설정
image_dir = r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_3w\original\mod\images\train_temp"
label_dir = r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_3w\original\mod\labels\train_temp"
image_ext = '.jpg'  # 또는 .png 등

total_area = 0
total_instances = 0

def imread_unicode(path):
    with open(path, 'rb') as f:
        image = np.asarray(bytearray(f.read()), dtype=np.uint8)
        return cv2.imdecode(image, cv2.IMREAD_COLOR)


for label_file in os.listdir(label_dir):
    if not label_file.endswith('.txt'):
        continue

    image_name = os.path.splitext(label_file)[0] + image_ext
    image_path = os.path.join(image_dir, image_name)

    if not os.path.exists(image_path):
        print(f"❗ 이미지 없음: {image_path}")
        continue

    image = imread_unicode(image_path)
    h, w = image.shape[:2]

    label_path = os.path.join(label_dir, label_file)
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7 or (len(parts) - 1) % 2 != 0:
            continue  # 다각형이 되려면 최소한 3개 점 필요

        class_id = int(parts[0])
        coords = list(map(float, parts[1:]))
        points = np.array([[int(float(x) * w), int(float(y) * h)] for x, y in zip(coords[::2], coords[1::2])], dtype=np.int32)

        if len(points) < 3:
            continue  # 면적 없는 객체 제외

        area = cv2.contourArea(points)
        total_area += area
        total_instances += 1

# 평균 면적 계산
avg_area = total_area / total_instances if total_instances > 0 else 0

print(f"📦 총 객체 수: {total_instances}")
print(f"🧮 전체 면적 합 (px²): {total_area:.2f}")
print(f"📏 객체당 평균 면적 (px²): {avg_area:.2f}")

In [ ]:
one_weeks_t = []
one_weeks_v = []
two_weeks_t = []
two_weeks_v = []
three_weeks_t = []
three_weeks_v = []

In [ ]:
three_weeks_t = [total_instances, total_area, avg_area]
three_weeks_t

In [ ]:
three_weeks = [x + y for x, y in zip(three_weeks_t, three_weeks_v)]

print(three_weeks_t)
print("===" * 20)
print(three_weeks_v)
print("===" * 20)
print(three_weeks)

In [ ]:
print(one_weeks)
print("===" * 20)
print(two_weeks)
print("===" * 20)
print(three_weeks)

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 유니코드 경로 이미지 로드 함수
def imread_unicode(path):
    with open(path, 'rb') as f:
        file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
        return cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

# 세트별 폴더 리스트 (2개씩 세트 구성)
folder_sets = [
    [
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_1w\original\mod\images\train_temp",  # <- 여기에 첫 번째 폴더 경로
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_1w\original\mod\images\val_temp"   # <- 여기에 두 번째 폴더 경로   
    ],
    [
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_2w\original\mod\images\train_temp",  # <- 여기에 첫 번째 폴더 경로
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_2w\original\mod\images\val_temp"   # <- 여기에 두 번째 폴더 경로
    ],
    [
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_3w\original\mod\images\train_temp",  # <- 여기에 첫 번째 폴더 경로
        r"F:\세종대학교\강의\박사1학기\생명과학을위한머신러닝응용및실습\mini_project\data\strawberry_3w\original\mod\images\val_temp"   # <- 여기에 두 번째 폴더 경로
    ]
]

image_ext = ['.jpg', '.jpeg', '.png']
colors = ['r', 'g', 'b']

# 세트 반복
for idx, folders in enumerate(folder_sets):
    hist_sum = np.zeros((3, 256), dtype=np.float64)
    image_count = 0

    for folder in folders:
        for fname in os.listdir(folder):
            if not any(fname.lower().endswith(ext) for ext in image_ext):
                continue

            img_path = os.path.join(folder, fname)
            image = imread_unicode(img_path)
            if image is None:
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            for i in range(3):
                hist = cv2.calcHist([image], [i], None, [256], [0, 256])
                hist_sum[i] += hist[:, 0]

            image_count += 1

    if image_count == 0:
        print(f"⚠️ {idx+1}w에는 유효한 이미지가 없습니다.")
        continue

    hist_avg = hist_sum / image_count

    # === 그래프 하나씩 출력 ===
    plt.figure(figsize=(6, 4))
    for i, color in enumerate(colors):
        plt.plot(hist_avg[i], color=color, label=f'{color.upper()} Channel')

    plt.title(f'{idx+1}w 평균 RGB 히스토그램')
    plt.xlabel("픽셀값 (0~255)")
    plt.ylabel("평균 빈도수")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()